# Automated Gleason Grading

## Imports

In [1]:
import os

import openslide
import random
import seaborn as sns
import cv2

# Torch packages
import torch
from torch.utils.data import Dataset
from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import functional as F
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
import math
import time
from IPython.display import Image, display
from tqdm import tqdm
# from sklearn.model_selection import train_test_split

# Plotly for the interactive viewer
import plotly.graph_objs as go

/home/danylenko/auto_gleason/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [3]:
# Location of the training images

DATA_PATH = '../../ganz/data/panda_dataset'

# image and mask directories
data_dir = f'{DATA_PATH}/train_images'
mask_dir = f'{DATA_PATH}/train_label_masks'


# Location of training labels
train = pd.read_csv(f'{DATA_PATH}/train.csv').set_index('image_id')
test = pd.read_csv(f'{DATA_PATH}/test.csv')
submission = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
# # determine if we will be pinning memory during data loading
# PIN_MEMORY = True if DEVICE == "cuda" else False

cuda


## Visualize Data

### Sample Images

In [4]:
# all_train_img_names = [
#     '07a7ef0ba3bb0d6564a73f4f3e1c2293',
#     '037504061b9fba71ef6e24c48c6df44d',
#     '035b1edd3d1aeeffc77ce5d248a01a53',
#     '059cbf902c5e42972587c8d17d49efed',
#     '06a0cbd8fd6320ef1aa6f19342af2e68',
#     '06eda4a6faca84e84a781fee2d5f47e1',
#     '0a4b7a7499ed55c71033cefb0765e93d',
#     '0838c82917cd9af681df249264d2769c',
#     '046b35ae95374bfb48cdca8d7c83233f',
#     '074c3e01525681a275a42282cd21cbde',
#     '05abe25c883d508ecc15b6e857e59f32',
#     '05f4e9415af9fdabc19109c980daf5ad',
#     '060121a06476ef401d8a21d6567dee6d',
#     '068b0e3be4c35ea983f77accf8351cc8',
#     '08f055372c7b8a7e1df97c6586542ac8'
# ]

all_train_img_names = list(train.index)
print(all_train_img_names[:10])

['0005f7aaab2800f6170c399693a96917', '000920ad0b612851f8e01bcc880d9b3d', '0018ae58b01bdadc8e347995b69f99aa', '001c62abd11fa4b57bf7a6c603a11bb9', '001d865e65ef5d2579c190a0e0350d8f', '002a4db09dad406c85505a00fb6f6144', '003046e27c8ead3e3db155780dc5498e', '0032bfa835ce0f43a92ae0bbab6871cb', '003a91841da04a5a31f808fb5c21538a', '003d4dd6bd61221ebc0bfb9350db333f']


### Display Images

In [20]:
def display_images(slides): 
    f, ax = plt.subplots(5,3, figsize=(18,22))
    for i, slide in enumerate(slides):
        image = openslide.OpenSlide(os.path.join(data_dir, f'{slide}.tiff'))
        # spacing = 1 / (float(image.properties['tiff.XResolution']) / 10000)
        patch = image.read_region((1780,1950), 0, (256, 256))
        ax[i//3, i%3].imshow(patch) 
        image.close()       
        ax[i//3, i%3].axis('off')
        
        image_id = slide
        data_provider = train.loc[slide, 'data_provider']
        isup_grade = train.loc[slide, 'isup_grade']
        gleason_score = train.loc[slide, 'gleason_score']
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")

    plt.show()

In [ ]:
display_images(all_train_img_names)

### Visualize Masks

In [21]:
def display_masks(slides): 
    f, ax = plt.subplots(5,3, figsize=(18,22))
    for i, slide in enumerate(slides):
        
        mask = openslide.OpenSlide(os.path.join(mask_dir, f'{slide}_mask.tiff'))
        mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
        cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])

        ax[i//3, i%3].imshow(np.asarray(mask_data)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=5) 
        mask.close()       
        ax[i//3, i%3].axis('off')
        
        image_id = slide
        data_provider = train.loc[slide, 'data_provider']
        isup_grade = train.loc[slide, 'isup_grade']
        gleason_score = train.loc[slide, 'gleason_score']
        ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")
        f.tight_layout()
        
    plt.show()

In [ ]:
display_masks(all_train_img_names)

## Patch Segmentation

### Initialize Globals

In [5]:
# define the number of channels in the input, number of classes,
# and number of levels in the U-Net model
NUM_CHANNELS = 1
NUM_CLASSES = 1
NUM_LEVELS = 3
# initialize learning rate, number of epochs to train for, and the batch size
INIT_LR = 0.001
NUM_EPOCHS = 40
BATCH_SIZE = 1 #64
# define the input image dimensions
BATCH_WIDTH = 256
BATCH_HEIGHT = 256
# define threshold to filter weak predictions
THRESHOLD = 0.5

# define the test split
TEST_SPLIT = 0.85

# define the path to the base output directory
BASE_OUTPUT = "../output"
# define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = f"{BASE_OUTPUT}/unet_tgs_salt.pth"
PLOT_PATH = f"{BASE_OUTPUT}/plot.png"
TEST_PATHS = f"{BASE_OUTPUT}/test_paths.txt"

### Create Training Set

In [6]:
def create_image_set(slides):
  imgs = []
  masks = []
  for i, slide in enumerate(slides):
    image = openslide.OpenSlide(os.path.join(data_dir, f'{slide}.tiff'))
    mask = openslide.OpenSlide(os.path.join(mask_dir, f'{slide}_mask.tiff'))
    # print("img level_count: " + str(image.level_count))
    # print("img level 0 dimension x: " + str(image.dimensions[0]))
    # print("img level 0 dimension y: " + str(image.dimensions[1]))
    # print("img level 1 dimension x: " + str(image.level_dimensions[1][0]))
    # print("img level 1 dimension y: " + str(image.level_dimensions[1][1]))
    # print("img level 2 dimension x: " + str(image.level_dimensions[2][0]))
    # print("img level 2 dimension y: " + str(image.level_dimensions[2][1]))
    # print("mask level 0 dimension x: " + str(mask.dimensions[0]))
    # print("mask level 0 dimension y: " + str(mask.dimensions[1]))
    max_x = image.dimensions[0] - (image.dimensions[0] % BATCH_WIDTH)
    max_y = image.dimensions[1] - (image.dimensions[1] % BATCH_HEIGHT)
    x = 0
    while x < max_x:
      y = 0
      while y < max_y:
        imgs.append(image.read_region((x,y), 0, (BATCH_WIDTH, BATCH_HEIGHT)))
        masks.append(mask.read_region((x,y), 0, (BATCH_WIDTH, BATCH_HEIGHT)))
        y+=BATCH_HEIGHT
      x+=BATCH_WIDTH
  return imgs, masks

In [24]:
create_image_set(all_train_img_names[1:2])

img level 0 dimension x: 10496
img level 0 dimension y: 24832
mask level 0 dimension x: 10496
mask level 0 dimension y: 24832


([], [])

### Dataset Class

In [7]:
class SegmentationDataset(Dataset):
  def __init__(self, wsi_list):
    self.wsi_list = wsi_list

  def __len__(self):
    # return the number of total samples contained in the dataset
    return len(self.wsi_list)

  def __getitem__(self, idx):
    # grab the image from the current index
    wsi_name = self.wsi_list[idx]
    wsi = openslide.OpenSlide(os.path.join(data_dir, f'{wsi_name}.tiff'))
    mask = openslide.OpenSlide(os.path.join(mask_dir, f'{wsi_name}_mask.tiff'))
    max_x = wsi.dimensions[0] - (wsi.dimensions[0] % BATCH_WIDTH)
    max_y = wsi.dimensions[1] - (wsi.dimensions[1] % BATCH_HEIGHT)
    imgs = []
    masks = []
    x = 0
    while x < max_x:
      y = 0
      while y < max_y:
        imgs.append(wsi.read_region((x,y), 0, (BATCH_WIDTH, BATCH_HEIGHT)))
        masks.append(mask.read_region((x,y), 0, (BATCH_WIDTH, BATCH_HEIGHT)))
        y+=BATCH_HEIGHT
      x+=BATCH_WIDTH
    # return the patch images and masks
    return imgs, masks

### UNET Modules

In [8]:
class Block(Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		# store the convolution and RELU layers
		self.conv1 = Conv2d(inChannels, outChannels, 3)
		self.relu = ReLU()
		self.conv2 = Conv2d(outChannels, outChannels, 3)
    
	def forward(self, x):
		# apply CONV => RELU => CONV block to the inputs and return it
		return self.conv2(self.relu(self.conv1(x)))

class Encoder(Module):
	def __init__(self, channels=(3, 16, 32, 64)):
		super().__init__()
		# store the encoder blocks and maxpooling layer
		self.encBlocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])
		self.pool = MaxPool2d(2)
    
	def forward(self, x):
		# initialize an empty list to store the intermediate outputs
		blockOutputs = []
		# loop through the encoder blocks
		for block in self.encBlocks:
			# pass the inputs through the current encoder block, store
			# the outputs, and then apply maxpooling on the output
			x = block(x)
			blockOutputs.append(x)
			x = self.pool(x)
		# return the list containing the intermediate outputs
		return blockOutputs

class Decoder(Module):
	def __init__(self, channels=(64, 32, 16)):
		super().__init__()
		# initialize the number of channels, upsampler blocks, and
		# decoder blocks
		self.channels = channels
		self.upconvs = ModuleList(
			[ConvTranspose2d(channels[i], channels[i + 1], 2, 2)
			 	for i in range(len(channels) - 1)])
		self.dec_blocks = ModuleList(
			[Block(channels[i], channels[i + 1])
			 	for i in range(len(channels) - 1)])

	def forward(self, x, encFeatures):
		# loop through the number of channels
		for i in range(len(self.channels) - 1):
			# pass the inputs through the upsampler blocks
			x = self.upconvs[i](x)
			# crop the current features from the encoder blocks,
			# concatenate them with the current upsampled features,
			# and pass the concatenated output through the current
			# decoder block
			encFeat = self.crop(encFeatures[i], x)
			x = torch.cat([x, encFeat], dim=1)
			x = self.dec_blocks[i](x)
		# return the final decoder output
		return x
    
	def crop(self, encFeatures, x):
		# grab the dimensions of the inputs, and crop the encoder
		# features to match the dimensions
		(_, _, H, W) = x.shape
		encFeatures = CenterCrop([H, W])(encFeatures)
		# return the cropped features
		return encFeatures
		
class UNet(Module):
	def __init__(self, encChannels=(3, 16, 32, 64),
		 decChannels=(64, 32, 16),
		 nbClasses=1, retainDim=True,
		 outSize=(BATCH_HEIGHT,  BATCH_WIDTH)):
		super().__init__()
		# initialize the encoder and decoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize the regression head and store the class variables
		self.head = Conv2d(decChannels[-1], nbClasses, 1)
		self.retainDim = retainDim
		self.outSize = outSize

	def forward(self, x):
		# grab the features from the encoder
		encFeatures = self.encoder(x)
		# pass the encoder features through decoder making sure that
		# their dimensions are suited for concatenation
		decFeatures = self.decoder(encFeatures[::-1][0],
			encFeatures[::-1][1:])
		# pass the decoder features through the regression head to
		# obtain the segmentation mask
		map = self.head(decFeatures)
		# check to see if we are retaining the original output
		# dimensions and if so, then resize the output to match them
		if self.retainDim:
			map = F.interpolate(map, self.outSize)
		# return the segmentation map
		return map

### Training

In [9]:
# all_train_imgs, all_train_masks = create_image_set(all_train_img_names)

# partition the data into training and validation splits using 85% of
# the data for training and the remaining 15% for validation
# split = train_test_split(all_train_imgs, all_train_masks,
# 	test_size=TEST_SPLIT, random_state=42)
# split = train_test_split(all_train_img_names,
# 	test_size=TEST_SPLIT, random_state=42)
split_size = math.floor(TEST_SPLIT*len(all_train_img_names))
split = torch.utils.data.random_split(all_train_img_names,
	[split_size, len(all_train_img_names)-split_size], 
	generator=torch.Generator().manual_seed(42))

# unpack the data split
# (train_imgs, val_imgs) = split[:2]
# (train_masks, val_masks) = split[2:]
(train_img_names, val_img_names) = split
train_img_names = list(train_img_names)
val_img_names = list(val_img_names)
print(train_img_names[:10])
print(val_img_names[:10])

['8b36508994d132ac03c5effd255b6eaf', 'aca61df8852daa183fa8cbf30a2e8660', 'a4fe6a765a5d525015f97f2f967caa67', '18e2d22113d3bf1ef0575c90cb22fde6', '037f29b7d62479c7fb257903f401269a', '0fe439d806b88e850a6a69442d295530', '2e512e923950da10c0b4432d08ca0663', 'ef528624c588c2f9d04c79d400f1aacc', 'd4f83e0d9b46e89cd5e9da401f625bce', 'cc3d4d77585447466f27904725bd1d7b']
['8a62c7e1f8070d642c7de7156dcf55bb', 'ce04c1a8d8577bb9620620c0a62edb15', '375771fed439d26b5374aae3f6b1d653', '0ec5bf6cbb0a9195b4db46173203a848', 'ac07fd6531d7536124ceab1990d5227d', 'f35124ad7e0c9d8537a8e57966c06445', 'e692db297233f27312315e8a5c75c32a', 'e3177c4430b780c973ef857a320a211d', '92d641eafe819ad90148027f5da9afd3', '5c3aa6266bfcbf2f9bf057a0a5e39520']


In [10]:
# create the train and validation datasets
trainDS = SegmentationDataset(wsi_list=train_img_names)
valDS = SegmentationDataset(wsi_list=val_img_names)
print(f"[INFO] found {len(trainDS)} wsi in the training set...")
print(f"[INFO] found {len(valDS)} wsi in the test set...")

# create the training and validation data loaders
trainLoader = DataLoader(trainDS, shuffle=True,
	batch_size=BATCH_SIZE, num_workers=os.cpu_count())
valLoader = DataLoader(valDS, shuffle=False,
	batch_size=BATCH_SIZE, num_workers=os.cpu_count())

[INFO] found 9023 wsi in the training set...
[INFO] found 1593 wsi in the test set...


In [11]:
# initialize our UNet model
unet = UNet().to(DEVICE)
# initialize loss function and optimizer
lossFunc = BCEWithLogitsLoss()
opt = Adam(unet.parameters(), lr=INIT_LR)
# # calculate steps per epoch for training and validation set
# trainSteps = len(trainDS) // BATCH_SIZE
# valSteps = len(valDS) // BATCH_SIZE
# initialize a dictionary to store training history
H = {"train_loss": [], "val_loss": []}

# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(NUM_EPOCHS)):
	# set the model in training mode
	unet.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	trainSteps = 0
	valSteps = 0

	# loop over the training set
	for (i, (x, y)) in enumerate(trainLoader):
		for (x_patch, y_patch) in zip(x, y):
			# send the input to the device
			(x_patch, y_patch) = (x_patch.to(DEVICE), y_patch.to(DEVICE))
			# perform a forward pass and calculate the training loss
			pred = unet(x_patch)
			loss = lossFunc(pred, y_patch)
			# first, zero out any previously accumulated gradients, then
			# perform backpropagation, and then update model parameters
			opt.zero_grad()
			loss.backward()
			opt.step()
			# add the loss to the total training loss so far
			totalTrainLoss += loss
			trainSteps += 1
    
	# switch off autograd
	with torch.no_grad():
		# set the model in evaluation mode
		unet.eval()
		# loop over the validation set
		for (x, y) in valLoader:
			for (x_patch, y_patch) in zip(x, y):
				# send the input to the device
				(x_patch, y_patch) = (x_patch.to(DEVICE), y_patch.to(DEVICE))
				# make the predictions and calculate the validation loss
				pred = unet(x_patch)
				totalValLoss += lossFunc(pred, y_patch)
				valSteps += 1

	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
	print("Train loss: {:.6f}, Val loss: {:.4f}".format(
		avgTrainLoss, avgValLoss))

# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))

[INFO] training the network...


  0%|          | 0/40 [04:34<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# plot the training loss
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)
# serialize the model to disk
torch.save(unet, MODEL_PATH)

## Testing

In [ ]:
def prepare_plot(origImage, origMask, predMask):
	# initialize our figure
	figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 10))
	# plot the original image, its mask, and the predicted mask
	ax[0].imshow(origImage)
	ax[1].imshow(origMask)
	ax[2].imshow(predMask)
	# set the titles of the subplots
	ax[0].set_title("Image")
	ax[1].set_title("Original Mask")
	ax[2].set_title("Predicted Mask")
	# set the layout of the figure and display it
	figure.tight_layout()
	figure.show()

def make_predictions(model, test_image, test_mask):
	# set model to evaluation mode
	model.eval()
	# turn off gradient tracking
	with torch.no_grad():
		# resize the image and make a copy of it for visualization
		# test_image = cv2.resize(test_image, (BATCH_WIDTH, BATCH_HEIGHT))
		orig = test_image.copy()

  	# make the channel axis to be the leading one, add a batch
		# dimension, create a PyTorch tensor, and flash it to the
		# current device
		test_image = np.transpose(test_image, (2, 0, 1))
		test_image = np.expand_dims(test_image, 0)
		test_image = torch.from_numpy(test_image).to(DEVICE)
		# make the prediction, pass the results through the sigmoid
		# function, and convert the result to a NumPy array
		predMask = model(test_image).squeeze()
		predMask = torch.sigmoid(predMask)
		predMask = predMask.cpu().numpy()
		# filter out the weak predictions and convert them to integers
		predMask = (predMask > THRESHOLD) * 255
		predMask = predMask.astype(np.uint8)
		# prepare a plot for visualization
		prepare_plot(orig, gtMask, predMask)

In [ ]:
# get test_img ids
test_img_names = test.image_id
print(test_img_names.head)

# load the image paths in our testing file and randomly select 2 image paths
print("[INFO] loading up test images...")
random_test_img_names = np.random.choice(test_img_names, size=2)
test_imgs, test_masks = create_image_set(random_test_img_names)

# load our model from disk and flash it to the current device
print("[INFO] load up model...")
unet = torch.load(MODEL_PATH).to(DEVICE)

# iterate over the randomly selected test image paths
for test_img, test_mask in zip(test_imgs, test_masks):
	# make predictions and visualize the results
	make_predictions(unet, test_img, test_mask)